# 1.
## (a) (5%) Define decision variables and give linear programming (LP) formulation with respect to average yield scenario (i.e. EV solution).

### set:
$I$ : set of products, $I=$ {wheat, corn, sugar}

### index:
$i$ : index of products $i$, $i$ $\in$ $I$

### parameters:
$C_i$ : cost per acre for each unit product $i$, $\forall$$i$ $\in$ $I$  
$Y_i$ : average yield of product $i$ per acre, $\forall$$i$ $\in$ $I$  
$D_i$ : price changing threshold for product $i$, $\forall$$i$ $\in$ $I$  
$P^a_i$ : price for product $i$ when its yield is above $D_i$, $\forall$$i$ $\in$ $I$  
$P^b_i$ : price for product $i$ when its yield is below $D_i$, $\forall$$i$ $\in$ $I$

### decision variables:
$l_i$ : lands to plant product $i$, $\forall$$i$ $\in$ $I$  
$p_i$ : profit gain when product $i$ harvest, $\forall$$i$ $\in$ $I$

### objective function:
$$max = \sum_{i\in I}p_i - \sum_{i\in I}C_i l_i$$
### constraints:
$$\sum_{i\in I}l_i \leq 500$$  
$$p_i \leq -|P^b_i| * D_i + |P^b_i| * l_i * Y_i, i={wheat, corn}$$  
$$p_i \leq -|P^a_i| * D_i + |P^a_i| * l_i * Y_i, i= {wheat, corn} $$  
$$p_i \leq P^b_i * l_i * Y_i, i={sugar}$$  
$$p_i \leq |P^b_i - P^a_i| * D_i + P^a_i * l_i * Y_i, i={sugar}$$  
$$l_i \geq 0, i \in I$$  
$$p_i \in free, i \in I$$  

In [1]:
from pulp import *
import numpy as np
import scipy.stats

In [2]:
def solve_prob(LpProb):
    LpProb.solve()
    #查看目前解的狀態
    print("Status:", LpStatus[LpProb.status])

    #印出解及目標值
    for v in LpProb.variables():
        print(v.name, "=", v.varValue)
    print('obj=',value(LpProb.objective))
    pass

In [3]:
## add parameters
Products = ['Wheat', 'Corn', 'Sugar']

Costs = { # per acre
    Products[0]: 150, 
    Products[1]: 230, 
    Products[2]: 260, 
    }

Avg_yield = { # average yield of products per acre
    Products[0]: 2.5, 
    Products[1]: 3, 
    Products[2]: 20, 
    }

Demands = { # ton
    Products[0]: 200, 
    Products[1]: 240, 
    Products[2]: 6000, # set sugar price threshhold as demand
    }

Price_above = { # selling price when total yeild is above demand
    Products[0]: 170, 
    Products[1]: 150, 
    Products[2]: 10, 
    }

Price_below = { # selling price when total yeild is below demand
    Products[0]: -238, 
    Products[1]: -210, 
    Products[2]: 36, 
    }

In [4]:
def prob_senario_ana(prob_name="problem_s", s=1):
    ## problem
    prob_senario = LpProblem("%s" %prob_name, LpMaximize)

    ## add variables
    land_vars = LpVariable.dicts(name="Land", indexs=Products, lowBound=0, upBound=None, cat="continuous")
    profits = LpVariable.dicts(name="profit", indexs=Products, lowBound=None, upBound=None, cat="continuous")

    ## objective function
    prob_senario += lpSum(
        [-Costs[i] * land_vars[i] for i in Products] + 
        [profits[i] for i in Products] # splitting piecewise profit function into two
        )

    ## constraints
    prob_senario += 500 >= lpSum(land_vars[i] for i in Products) # total land limit

    prob_senario += profits["Wheat"] <= [-np.abs(Price_below[i]) * Demands[i] + np.abs(Price_below[i]) * land_vars[i] * Avg_yield[i] * s for i in ["Wheat"]] # pay price when yield below demand (piecewise)
    prob_senario += profits["Wheat"] <= [-np.abs(Price_above[i]) * Demands[i] + np.abs(Price_above[i]) * land_vars[i] * Avg_yield[i] * s for i in ["Wheat"]] # get profit when yield above demand (piecewise)

    prob_senario += profits["Corn"] <= [-np.abs(Price_below[i]) * Demands[i] + np.abs(Price_below[i]) * land_vars[i] * Avg_yield[i] * s for i in ["Corn"]] # pay price when yield below demand (piecewise)
    prob_senario += profits["Corn"] <= [-np.abs(Price_above[i]) * Demands[i] + np.abs(Price_above[i]) * land_vars[i] * Avg_yield[i] * s for i in ["Corn"]] # get profit when yield above demand (piecewise)

    prob_senario += profits["Sugar"] <= [Price_below[i] * land_vars[i] * Avg_yield[i] * s for i in ["Sugar"]] # get higher profit ratio when yield below demand (piecewise)
    prob_senario += profits["Sugar"] <= [np.abs(Price_below[i] - Price_above[i]) * Demands[i] + Price_above[i] * land_vars[i] * Avg_yield[i] * s for i in ["Sugar"]] # get lower profit ratio when yield above demand (piecewise)

    ## solve
    prob_senario.solve()
    return prob_senario

In [5]:
def prob_result(prob):
    print("Status for %s:" %prob.name, LpStatus[prob.status])

    #印出解及目標值
    for v in prob.variables():
        print(v.name, "=", v.varValue)
    print('obj=',value(prob.objective))
    pass

In [6]:
def senario_obj(determined_land, s=1):
    cost = np.sum([determined_land[i] * Costs[i] for i in Products])

    profits = {
        Products[0]: 0,
        Products[1]: 0,
        Products[2]: 0,
    }

    for i in Products:
        y = determined_land[i] * Avg_yield[i] * s
        if y <= Demands[i]:
            if i != "Sugar":
                profits[i] = Price_below[i] * (Demands[i] - y)
            else:
                profits[i] = Price_below[i] * y
        else:
            if i != "Sugar":
                profits[i] = Price_above[i] * (y - Demands[i])
            else:
                profits[i] = Price_above[i] * (y - Demands[i]) + Price_below[i] * Demands[i]
    return np.sum([profits[i] for i in Products]) - cost

In [21]:
## 1a
prob_a = prob_senario_ana(prob_name="EV_solution")
prob_result(prob=prob_a)

# https://docs.mosek.com/modeling-cookbook/mio.html
# http://civil.colorado.edu/~balajir/CVEN5393/lectures/chapter-08.pdf

Status for EV_solution: Optimal
Land_Corn = 80.0
Land_Sugar = 300.0
Land_Wheat = 120.0
profit_Corn = 0.0
profit_Sugar = 216000.0
profit_Wheat = 17000.0
obj= 118600.0


## (b) (5%) Solve the solution of Scenario Analysis and the EV solution?

In [8]:
## 1b
Senario = {
    "high" : 1.2, 
    "avg" : 1, 
    "low" : 0.8
    }

In [22]:
prob_1bs = {}
for s in Senario:
    prob_1bs[s] = prob_senario_ana(prob_name="senario_analysis_%s" %s, s=Senario[s])
    prob_result(prob=prob_1bs[s])
    print()


Status for senario_analysis_high: Optimal
Land_Corn = 66.666667
Land_Sugar = 250.0
Land_Wheat = 183.33333
profit_Corn = 0.0
profit_Sugar = 216000.0
profit_Wheat = 59500.0
obj= 167666.66709

Status for senario_analysis_avg: Optimal
Land_Corn = 80.0
Land_Sugar = 300.0
Land_Wheat = 120.0
profit_Corn = 0.0
profit_Sugar = 216000.0
profit_Wheat = 17000.0
obj= 118600.0

Status for senario_analysis_low: Optimal
Land_Corn = 25.0
Land_Sugar = 375.0
Land_Wheat = 100.0
profit_Corn = -37800.0
profit_Sugar = 216000.0
profit_Wheat = 0.0
obj= 59950.0



### Answer for (b):
#### senario analysis:
- objective value when high demand: 167666
- objective value when average demand: 118600
- objective value when low demand: 59950  

#### EV solution:
- objective value: 118600

## (c) (5%) Define new decision variables and give deterministic equivalent problem (DEP) formulation of two-stage recourse problem (RP).

### scenario:
for each scenario $\omega$ $\in$ {$\omega^1$, $\omega^2$, $\omega^3$}, where each correspond to scenarios "high", "average", "low" for yield rate

### index:
$s$ : index of scenarios $s$, $s$ $\in$ $\omega$

### add parameters:
$S_s$ : yield rate scenario for $s$, $\forall$$s$ $\in$ $\omega$  

### first-stage decision variables:
$l_i$ : lands to plant product $i$, $\forall$$i$ $\in$ $I$ 

### second-stage decision variables:
$p^s_i$ : profit gain when product $i$ harvest under scenario $s$, $\forall$$i$ $\in$ $I$, $\forall$$s$ $\in$ $\omega$

### objective function:
$$max = 1/3 \sum_{s\in\omega}\sum_{i\in I}p_i - \sum_{i\in I}C_i l_i$$
### constraints:
$$\sum_{i\in I}l_i \leq 500$$  
$$p_i \leq -|P^b_i| * D_i + |P^b_i| * l_i * Y_i * S_s, i=(wheat, corn), s\in\omega$$  
$$p_i \leq -|P^a_i| * D_i + |P^a_i| * l_i * Y_i * S_s, i= (wheat, corn), s\in\omega $$  
$$p_i \leq P^b_i * l_i * Y_i * S_s, i=(sugar), s\in\omega$$  
$$p_i \leq |P^b_i - P^a_i| * D_i + P^a_i * l_i * Y_i * S_s, i=(sugar), s\in\omega$$  
$$l_i \geq 0, i \in I$$  
$$p_i \in free, i \in I$$  
$$s\in\omega$$

## (d) (5%) Solve the RP model.

In [10]:
## 1d
## model
all_stages = LpProblem("problem_1c", LpMaximize)
## vars
lands = LpVariable.dicts(name="lands_for", indexs=Products, lowBound=0, upBound=None, cat="continuous")
profits = {}
for s in Senario:
    profits[s] = LpVariable.dicts(name="profit_when_%s_yield" %s, indexs=Products, lowBound=None, upBound=None, cat="continuous")

## obj
all_stages += lpSum([-Costs[i] * lands[i] for i in Products]) + 1/3 * lpSum([profits[s][i] for s in Senario for i in Products])

## st
### land
all_stages += 500 >= lpSum([lands[i] for i in Products])

### 
for s in Senario:
    all_stages += profits[s]["Wheat"] <= [-np.abs(Price_below[i]) * Demands[i] + np.abs(Price_below[i]) * lands[i] * Avg_yield[i] * Senario[s] for i in ["Wheat"]] # pay price when yield below demand (piecewise)
    all_stages += profits[s]["Wheat"] <= [-np.abs(Price_above[i]) * Demands[i] + np.abs(Price_above[i]) * lands[i] * Avg_yield[i] * Senario[s] for i in ["Wheat"]] # get profit when yield above demand (piecewise)

    all_stages += profits[s]["Corn"] <= [-np.abs(Price_below[i]) * Demands[i] + np.abs(Price_below[i]) * lands[i] * Avg_yield[i] * Senario[s] for i in ["Corn"]] # pay price when yield below demand (piecewise)
    all_stages += profits[s]["Corn"] <= [-np.abs(Price_above[i]) * Demands[i] + np.abs(Price_above[i]) * lands[i] * Avg_yield[i] * Senario[s] for i in ["Corn"]] # get profit when yield above demand (piecewise)

    all_stages += profits[s]["Sugar"] <= [Price_below[i] * lands[i] * Avg_yield[i] * Senario[s] for i in ["Sugar"]] # get higher profit ratio when yield below demand (piecewise)
    all_stages += profits[s]["Sugar"] <= [np.abs(Price_below[i] - Price_above[i]) * Demands[i] + Price_above[i] * lands[i] * Avg_yield[i] * Senario[s] for i in ["Sugar"]] # get lower profit ratio when yield above demand (piecewise)

solve_prob(all_stages)

Status: Optimal
lands_for_Corn = 80.0
lands_for_Sugar = 250.0
lands_for_Wheat = 170.0
profit_when_avg_yield_Corn = 0.0
profit_when_avg_yield_Sugar = 180000.0
profit_when_avg_yield_Wheat = 38250.0
profit_when_high_yield_Corn = 7200.0
profit_when_high_yield_Sugar = 216000.0
profit_when_high_yield_Wheat = 52700.0
profit_when_low_yield_Corn = -10080.0
profit_when_low_yield_Sugar = 144000.0
profit_when_low_yield_Wheat = 23800.0
obj= 108390.0


### lands for each product:
- wheat: 170
- corn: 80
- sugar: 250

In [24]:
land_1d = {}
land_1d[Products[0]] = all_stages.variables()[2].varValue
land_1d[Products[1]] = all_stages.variables()[0].varValue
land_1d[Products[2]] = all_stages.variables()[1].varValue
for s in Senario:
    print("Recourse Solution profit when %s demand: " % s , senario_obj(determined_land=land_1d, s=Senario[s]))
    print()

Recourse Solution profit when high demand:  167000.0

Recourse Solution profit when avg demand:  109350.0

Recourse Solution profit when low demand:  48820.0



## (e) (5%) What’s the Expected Value of Perfect Information (EVPI)? What’s the value of the stochastic solution (VSS)?

In [25]:
## 1e EVPI, VSS
### EVPI
### max: WS - RP
ws = np.mean([value(prob_1bs[s].objective) for s in Senario])
rp = value(all_stages.objective)
evpi = ws - rp
### VSS
### max: EEV - RP
eev = 0
land_1a = {}
land_1a[Products[0]] = prob_a.variables()[2].varValue
land_1a[Products[1]] = prob_a.variables()[0].varValue
land_1a[Products[2]] = prob_a.variables()[1].varValue

for s in Senario:
    obj = senario_obj(determined_land=land_1a, s=Senario[s])
    print("EV for %s demand: " % s , obj)
    eev += (1/3) * (obj)
    print()
vss = rp - eev

EV for high demand:  148000.0

EV for avg demand:  118600.0

EV for low demand:  55120.0



In [27]:
print("EEV: ", round(eev))
print("VSS: ", round(vss))

EEV:  107240.0
VSS:  1150.0


## (f) (5%) Do you think RP providing a good solution in this study? Why?

#### yes, since EEV is positive, and VSS is not close to zero, we can suggest RP provide a better solution here

## (g) (15%) For continuous scenarios, if the weather results in a continuous probability distribution of the yield rate, which follows the normal distribution N(1,0.1). Calculate the objective function (i.e. expected value of the resource function) with the confidence intervals of the upper bound and lower bound, respectively, by Monte Carlo sampling average approximation (SAA). In the Monte Carlo simulation, we use the sample size 𝑁=𝑁̅=30 and the replications (batches) 𝑀=𝑇=15.

以下假設設定下限時皆是已經確定產出比率的情況下決定種植多少產品，所以都是採用 scenario analysis 的方法進行

In [13]:
## 1g
## set samples
mu, sigma = 1, 0.1
M = []
T = []
for i in range(15):
    M.append(np.random.normal(mu, sigma, 30))
    T.append(np.random.normal(mu, sigma, 30))

In [14]:
## lower
low_vars = []
low_obj = []
for i in range(15):
    low_vars.append([None] * 30)
    low_obj.append([None] * 30)

for i in range(15):
    for j in range(30):
        low_prob = prob_senario_ana(M[i][j])
        low_vars[i][j], low_obj[i][j] = low_prob.variables(), value(low_prob.objective)

In [15]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [16]:
lower_bound = {}
lower_bound["mean"], lower_bound["low"], lower_bound["high"] = mean_confidence_interval(data=[low_obj[i][j] for i in range(15) for j in range(30)])

we use mean of the decision variables of lower bound to calculate upper bound

In [17]:
## upper
land_mu = {}
land_mu["Wheat"] = np.mean([low_vars[i][j][2].varValue for i in range(15) for j in range(30)])
land_mu["Corn"] = np.mean([low_vars[i][j][0].varValue for i in range(15) for j in range(30)])
land_mu["Sugar"] = np.mean([low_vars[i][j][1].varValue for i in range(15) for j in range(30)])

In [18]:
up_obj = []
for i in range(15):
    up_obj.append([None] * 30)

for i in range(15):
    for j in range(30):
        up_obj[i][j] = senario_obj(determined_land=land_mu, s=T[i][j])

In [19]:
upper_bound = {}
upper_bound["mean"], upper_bound["low"], upper_bound["high"] = mean_confidence_interval(data=[up_obj[i][j] for i in range(15) for j in range(30)])

In [28]:
print("lower bound interval: ", lower_bound)
print("upper bound interval: ", upper_bound)

lower bound interval:  {'mean': 118600.0, 'low': 118600.0, 'high': 118600.0}
upper bound interval:  {'mean': 111847.9942440623, 'low': 109687.79893589333, 'high': 114008.18955223127}


After we plug in decision variables of lower bound into upper bound, we found that upper bound in this problem is lower than lower bound. It's beacuse this problem is a Maximun problem.

# 2

## (a) (5%) How many decision nodes are required? How many branches come out of each decision node?

one decision node, three branches out of decision node

## (b) (5%) How many chance nodes are required? How many branches come out of each chance node?

three chance node, two branches out of each chance node

## (c) (5%) Draw the decision tree. Label each branch completely including probabilities and payoffs.

![](https://i.imgur.com/7QF8iY8.jpg)

## (d) (5%) Solve the decision tree and find the best production strategy.

In [39]:
high = 0.41
low = 0.59
## A
a = 1000000 * high + (-400000) * low
## B
b = 600000 * high + (300000) * low
## C
c = 100000 * high + (400000) * low

print(a)
print(b)
print(c)

ev = np.max([a, b, c])
print("solution is b: ", ev)

174000.0
423000.0
277000.0
solution is b:  423000.0


## (e) (5%) Calculate the joint probabilities, i.e. P(X=0,θ =0), P(X=0,θ =1), P(X=1,θ =0), and P(X=1,θ=1).

In [47]:
hihi = 0.8
lolo = 0.7
p00 = lolo * low
p01 = (1-hihi) * high
p10 = (1-lolo) * low
p11 = hihi * high
print("P(X=0,θ =0): %f" %p00)
print("P(X=0,θ =1): %f" %p01)
print("P(X=1,θ =0): %f" %p10)
print("P(X=1,θ =1): %f" %p11)

P(X=0,θ =0): 0.413000
P(X=0,θ =1): 0.082000
P(X=1,θ =0): 0.177000
P(X=1,θ =1): 0.328000


## (f) (5%) Calculate the marginal probabilities, i.e. P(X=1) and P(X=0).

In [51]:
print("P(X=0): %f" %(p00 + p01))
print("P(X=1): %f" %(p10 + p11))

P(X=0): 0.495000
P(X=1): 0.505000


## (g) (5%) Calculate the posterior probabilities, i.e. P(θ =0|X=0), P(θ =1|X=0), P(θ =0|X=1), and P(θ=1|X=1).

In [54]:
print("P(θ =0|X=0): %f" %(p00 / (p00 + p01)))
print("P(θ =1|X=0): %f" %(p01 / (p00 + p01)))
print("P(θ =0|X=1): %f" %(p10 / (p10 + p11)))
print("P(θ =1|X=1): %f" %(p11 / (p10 + p11)))

P(θ =0|X=0): 0.834343
P(θ =1|X=0): 0.165657
P(θ =0|X=1): 0.350495
P(θ =1|X=1): 0.649505


## (h) (5%) Redraw the revised decision tree with considering two alternatives: Hire or Non-Hire marketing research firm.

![](https://i.imgur.com/FcbMs83.jpg)

## (i) (5%) Calculate the EVPI.

In [56]:
erpi = high * 1000000 + low * 400000
print("ERPI =", erpi)
print("ERw/o =", ev)
print("EVPI =", erpi - ev)

ERPI = 646000.0
ERw/o = 423000.0
EVPI = 223000.0


## (j) (5%) Calculate the EVE when we directly use the marketing results, i.e. if X=1 then Strategy A; otherwise X=0 then Strategy C.

In [57]:
ere = 0.505 * (0.65 * 1000000 + 0.35 * (-400000)) + 0.495 * (0.166 * 100000 + 0.834 * 400000)
print("ERE =", ere)
print("EVE =", ere - ev)

ERE = 430899.0
EVE = 7899.0


## (k) (5%) If hiring marketing research firm costs NTD 50,000, do you suggest NTU President to hire?

No, since value of the the perfect information is 7899, which is less than 50000. The cost of the research may decrease the expected value.